<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


## Adversarial Search (25 Points)

HW3 :: Practical Question 1

<br>

Corresponding TA: Aryan Ahadinia, Saeed Hedayatiyan


Consider the given $8\times8$ board game. There are two players in this game and they try to block each other and limit their valid moves by making walls. Their moves are limited to the 8 cells (or less when they are around the edges) beside them and the walls will start to appear only when a player steps into a random generated yellow cell and the wall will continue to be built until the player reaches the next yellow cell. The players start the game at $(0, 0)$ and $(7, 7)$ and they keep playing until one of them loses. There are multiple ways for a player to win the match:

- If a player steps into a wall or out of the board, the player loses the match!
- If the players reach the maximum moves the game finishes and the winner would be chosen based on the maximum valid move they have left plus the number of walls they have made and "No more moves" massege will appear on terminal.
- If the getMove() function does not return any value after 2 seconds, the player loses the match and "Time limit exceeded" massage will appear on terminal.

<center>
<img src="assets/Q3.png" style="width: 50%"/>
</center>


In this question, the codes for starting the game were provided for you in Helper-codes folder. You are expected to implement the MinimaxPlayer class (which inherit the Player class) with alpha-beta pruning.

In [1]:
%pip install tk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import random
import time
import math
from copy import deepcopy

from Helper_codes.graphics import *
from Helper_codes.question3 import Cell
from Helper_codes.question3 import IntPair
from Helper_codes.question3 import Player
from Helper_codes.question3 import NaivePlayer
from Helper_codes.question3 import Board
from Helper_codes.question3 import Game


You can run cell below and watch two naive players.

In [3]:
p1 = NaivePlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

numberOfMatches = 1
score1, score2 = g.start(numberOfMatches)

print(score1 / numberOfMatches)


Player 1 has exceeded the time limit
Player 2 has won

0.0


In [6]:
class MinimaxPlayer(Player):
    final_action = IntPair(-10, -10)
    def __init__(self, col, x, y, max_depth=4):
        super().__init__(col, x, y)
        self.max_depth = max_depth

    ##############################
    ## Feel free to add methods ##
    ##############################
    def possible_actions(self, board):
        actions = []
        size = board.getSize()
        x = board.getPlayerX(self.getCol())
        y = board.getPlayerY(self.getCol())
        xx = board.getPlayerX(3 - self.getCol())
        yy = board.getPlayerY(3 - self.getCol())
        
        if (x + 1 < size and y + 1 < size and (board.getCell(x + 1, y + 1).getColor() == 0 or board.getCell(x + 1, y + 1).getColor() == 3) and (x + 1 != xx or y + 1 != yy)):
            actions.append(IntPair(x + 1, y + 1))
        if (x + 1 < size and (board.getCell(x + 1, y).getColor() == 0 or board.getCell(x + 1, y).getColor() == 3) and (x + 1 != xx or y != yy)):
            actions.append(IntPair(x + 1, y))
        if (y + 1 < size and (board.getCell(x, y + 1).getColor() == 0 or board.getCell(x, y + 1).getColor() == 3) and (x != xx or y + 1 != yy)):
            actions.append(IntPair(x, y + 1))
        if (x - 1 >= 0 and y - 1 >= 0 and (board.getCell(x - 1, y - 1).getColor() == 0 or board.getCell(x - 1, y - 1).getColor() == 3) and (x - 1 != xx or y - 1 != yy)):
            actions.append(IntPair(x - 1, y - 1))
        if (x - 1 >= 0 and (board.getCell(x - 1, y).getColor() == 0 or board.getCell(x - 1, y).getColor() == 3) and (x - 1 != xx or y != yy)):
            actions.append(IntPair(x - 1, y))
        if (y - 1 >= 0 and (board.getCell(x, y - 1).getColor() == 0 or board.getCell(x, y - 1).getColor() == 3) and (x != xx or y - 1 != yy)):
            actions.append(IntPair(x, y - 1))
        if (x + 1 < size and y - 1 >= 0 and (board.getCell(x + 1, y - 1).getColor() == 0 or board.getCell(x + 1, y - 1).getColor() == 3) and (x + 1 != xx or y - 1 != yy)):
            actions.append(IntPair(x + 1, y - 1))
        if (x - 1 >= 0 and y + 1 < size and (board.getCell(x - 1, y + 1).getColor() == 0 or board.getCell(x - 1, y + 1).getColor() == 3) and (x - 1 != xx or y + 1 != yy)):
            actions.append(IntPair(x - 1, y + 1))
        return actions
    
    def min_value(self, board, alpha, beta, depth):
        ##########################
        # Code Here! (10 Points) #
        # Use alpha-beta pruning #
        ##########################
        if (depth > self.max_depth):
            return board.getScore(self.getCol())
        v = math.inf
        actions = self.possible_actions(board)
        for action in actions:
            copy_board = deepcopy(board)
            copy_board.move(action, self.getCol())
            v_unchanged = v
            v = min(v, self.max_value(copy_board, alpha, beta, depth + 1))
            if v != v_unchanged:
                self.final_action = action
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    def max_value(self, board, alpha, beta, depth):
        ##########################
        # Code Here! (10 Points) #
        # Use alpha-beta pruning #
        ##########################
        if (depth > self.max_depth):
            return board.getScore(self.getCol())
        v = -math.inf
        actions = self.possible_actions(board)
        for action in actions:
            copy_board = deepcopy(board)
            copy_board.move(action, self.getCol())
            v_unchanged = v
            v = max(v, self.min_value(copy_board, alpha, beta, depth + 1))
            if v != v_unchanged:
                self.final_action = action
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def getMove(self, board):
        ################################################################################
        # Code Here! (5 Points)                                                       #
        # return IntPair(-10, -10) when time limit exceeded or there is no possible move #
        # return IntPair(20, 20) when max number of moves exceeded                     #
        ################################################################################
        possible_actions_1 = self.possible_actions(board)
        if (len(possible_actions_1) == 0):
            return IntPair(-10, -10)
        
        start = time.time()
        v = self.max_value(deepcopy(board), -math.inf, math.inf, 0)
        
        
        if (board.getNumberOfMoves() > board.maxNumberOfMoves):
            return IntPair(20, 20) 
        
        
        return self.final_action



In [8]:
p1 = MinimaxPlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

number_of_matches = 10
score1, score2 = g.start(number_of_matches)

print(score1 / number_of_matches)


0.1
